In [1]:
import sys
sys.path.append('../')

In [2]:
import numpy as np
import tensorflow as tf
tf.keras.backend.set_floatx("float64")
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [3]:
from influence.influence_model import InfluenceModel

In [4]:
mnist_dataset = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist_dataset.load_data()

train_images = train_images / 255.0
test_images = test_images / 255.0

categorical_train_labels = tf.keras.utils.to_categorical(train_labels)
categorical_test_labels = tf.keras.utils.to_categorical(test_labels)

In [50]:
num_training_points = 54000

In [57]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(28, 28)),
#         tf.keras.layers.Dense(10, kernel_regularizer="l2", bias_regularizer="l2"),
        tf.keras.layers.Dense(10),
    ]
)

model.compile(
    optimizer="adam",
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["accuracy", tf.keras.metrics.CategoricalCrossentropy(from_logits=True)],
)

model.fit(
    train_images[:num_training_points],
    categorical_train_labels[:num_training_points],
    epochs=10,
    shuffle=False
)

Epoch 1/10
1688/1688 [==============================] - 1s 733us/step - loss: 0.4944 - accuracy: 0.8706 - categorical_crossentropy: 0.4944
Epoch 2/10
1688/1688 [==============================] - 1s 756us/step - loss: 0.3190 - accuracy: 0.9107 - categorical_crossentropy: 0.3190
Epoch 3/10
1688/1688 [==============================] - 1s 758us/step - loss: 0.2964 - accuracy: 0.9170 - categorical_crossentropy: 0.2964
Epoch 4/10
1688/1688 [==============================] - 1s 764us/step - loss: 0.2851 - accuracy: 0.9206 - categorical_crossentropy: 0.2851
Epoch 5/10
1688/1688 [==============================] - 1s 747us/step - loss: 0.2778 - accuracy: 0.9228 - categorical_crossentropy: 0.2778
Epoch 6/10
1688/1688 [==============================] - 1s 771us/step - loss: 0.2726 - accuracy: 0.9246 - categorical_crossentropy: 0.2726
Epoch 7/10
1688/1688 [==============================] - 1s 743us/step - loss: 0.2686 - accuracy: 0.9259 - categorical_crossentropy: 0.2686
Epoch 8/10
1688/1688 [=====

In [58]:
influence_model = InfluenceModel(
    model,
    train_images[:num_training_points],
    categorical_train_labels[:num_training_points],
    test_images,
    categorical_test_labels,
    model.loss,
    damping=0.2,
    dtype=np.float64,
)

influence_on_loss = influence_model.get_influence_on_loss(0, 0)
pred_loss_change = -(1/num_training_points) * influence_on_loss

In [59]:
new_model = tf.keras.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(28, 28)),
#         tf.keras.layers.Dense(10, kernel_regularizer="l2", bias_regularizer="l2"),
        tf.keras.layers.Dense(10),
    ]
)

new_model.compile(
    optimizer="adam",
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

training_idxs = np.delete(range(num_training_points), 0)

new_model.fit(
    train_images[training_idxs],
    categorical_train_labels[training_idxs],
    epochs=10,
    shuffle=False
)

Epoch 1/10
1688/1688 [==============================] - 1s 740us/step - loss: 0.4965 - accuracy: 0.8701
Epoch 2/10
1688/1688 [==============================] - 1s 726us/step - loss: 0.3190 - accuracy: 0.9111
Epoch 3/10
1688/1688 [==============================] - 1s 701us/step - loss: 0.2963 - accuracy: 0.9175
Epoch 4/10
1688/1688 [==============================] - ETA: 0s - loss: 0.2837 - accuracy: 0.92 - 1s 713us/step - loss: 0.2850 - accuracy: 0.9210
Epoch 5/10
1688/1688 [==============================] - 1s 708us/step - loss: 0.2777 - accuracy: 0.9232
Epoch 6/10
1688/1688 [==============================] - 1s 737us/step - loss: 0.2725 - accuracy: 0.9245
Epoch 7/10
1688/1688 [==============================] - 1s 700us/step - loss: 0.2685 - accuracy: 0.9259
Epoch 8/10
1688/1688 [==============================] - 1s 701us/step - loss: 0.2653 - accuracy: 0.9269
Epoch 9/10
1688/1688 [==============================] - 1s 701us/step - loss: 0.2626 - accuracy: 0.9278
Epoch 10/10
1688/1688 

In [60]:
original_loss = model.loss(categorical_test_labels[0:1], model(test_images[0:1]))
new_loss = new_model.loss(categorical_test_labels[0:1], new_model(test_images[0:1]))
actual_loss_change = new_loss - original_loss

In [61]:
print("Predicted Change in Loss: {}, Actual Change in Loss: {}".format(pred_loss_change, actual_loss_change))

Predicted Change in Loss: 8.423676880180541e-07, Actual Change in Loss: 9.24915075302124e-05
